In [1]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine
from ansys.mapdl.core.errors import MapdlRuntimeError
from ansys.mapdl.core import launch_mapdl
import re
import os
import pandas as pd
pd.set_option('display.max_colwidth', 150)

In [2]:
material_yielding_stress = 355
p_u = 3479
MAPDL_RUN_LOCATION='D:/constructal_automate_analysis_files'
MAPDL_START_TIMEOUT=120

In [8]:
mapdl = launch_mapdl(
    port=50052,
    run_location=MAPDL_RUN_LOCATION,
    nproc=4,
    override=True,
    loglevel="INFO",
    start_timeout=MAPDL_START_TIMEOUT,
    cleanup_on_exit=True
)

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - /SHOW SWITCH PLOTS TO  PNG         - RASTER MODE.
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - /SHOW SWITCH PLOTS TO  PNG         - RASTER MODE.
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files


In [4]:
def calc_post_processing_frequency_values_arr_position_for_ultimate_load(mapdl, p_u, flag):
    first_to_last_arr_position = len(mapdl.post_processing.frequency_values)-2-flag
    first_to_last_load = mapdl.post_processing.frequency_values[first_to_last_arr_position]

    second_to_last_arr_position = len(mapdl.post_processing.frequency_values)-3-flag
    second_to_last_load = mapdl.post_processing.frequency_values[second_to_last_arr_position]

    diff_between_loads = first_to_last_load - second_to_last_load
    load_increment = p_u/400

    if diff_between_loads > load_increment:
        return first_to_last_arr_position
    else:
        return second_to_last_arr_position

In [16]:
annotations_tensoes = {
    0.0: "0,00",
    88.75: "88,75",
    177.50: "177,50",
    266.25: "266,25",
    355.00: "355,00"
}

# annotations_tensoes = {
#     9.53: "9,53",
#     23.57: "23,57",
#     37.61: "37,61",
#     51.65: "51,65",
#     65.70: "65,70"
# }

def interagir_com_mapdl(mapdl, analysis_dir, analysis_file_name, analysis_db_path, flag, annotations_deslocamentos, suffix):

    # Abrir a pasta correta
    mapdl.cwd(analysis_dir)
    mapdl.filname(fname=analysis_file_name, key=0)
    mapdl.resume(fname=analysis_file_name, ext = 'db')

    # Calcular Carga e Tensão
    position_n_u = calc_post_processing_frequency_values_arr_position_for_ultimate_load(mapdl, p_u, flag)
    n_u = mapdl.post_processing.frequency_values[position_n_u]
    sigma_u = n_u/float(9.8)

    #Calcular Deslocamento Máximo
    position_w_max = position_n_u + 1
    mapdl.post1()
    mapdl.set(lstep=1, sbstep=position_w_max)
    negative_z_deflection = min(mapdl.post_processing.nodal_displacement("Z"))
    abs_negative_z_deflection = abs(negative_z_deflection)

    positive_z_deflection = max(mapdl.post_processing.nodal_displacement("Z"))

    if abs_negative_z_deflection > positive_z_deflection:
        z_deflection = abs_negative_z_deflection
    else:
        z_deflection = positive_z_deflection
    mapdl.finish()

    # Gerar imagens
    # VON_MISES_IMG_SUFFIX = '_von_mises_dist_detailed_Nts.png'
    # VON_MISES_IMG_SUFFIX = '_von_mises_dist_detailed_Nls.png'
    # VON_MISES_IMG_SUFFIX = '_von_mises_dist_deformado.png'
    # W_IMG_SUFFIX = '_w_dist_deformado.png'
    VON_MISES_IMG_SUFFIX = '_von_mises_dist_'+ suffix +'.png'
    W_IMG_SUFFIX = '_w_dist_'+ suffix +'.png'
    
    von_mises_dist_img_path = analysis_db_path.replace('.db', VON_MISES_IMG_SUFFIX)
    w_dist_img_path = analysis_db_path.replace('.db', W_IMG_SUFFIX)

    mapdl.result.plot_principal_nodal_stress(
        position_n_u,
        "SEQV",
        lighting=False,
        cpos="iso",
        background="white",
        text_color="black",
        add_text=False,
        show_edges=True,          
        edge_color="black",       
        cmap="jet",              
        # rng=[0, material_yielding_stress],
        line_width=1.0,           
        off_screen=True,
        # screenshot=von_mises_dist_img_path,
        show_scalar_bar=True,
        # show_displacement=True,
        # displacement_factor=20,
        scalar_bar_args={
            "title": "Tensões (MPa)",
            "vertical": False,
            # "fmt":"%8.2f",
            "width": 0.4,
            "position_x": 0.55,
            "position_y": 0.15,
            "title_font_size": 20,
            "label_font_size": 16,
            "n_labels": 0
        }
    )

    # Plotar tensões principais nodais
    plotter = mapdl.result.plot_principal_nodal_stress(
        position_n_u,
        "SEQV",
        lighting=False,
        cpos="iso",
        background="white",
        text_color="black",
        add_text=False,
        show_edges=True,
        edge_color="black",
        cmap="jet",
        rng=[0, material_yielding_stress],
        # rng=[9.53, 65.70],
        line_width=1.0,
        off_screen=True,
        screenshot=von_mises_dist_img_path,
        show_scalar_bar=True,
        show_displacement=True,
        # displacement_factor=20,
        displacement_factor=2,
        scalar_bar_args={
            "title": "Tensões (MPa)",
            "vertical": False,
            # "fmt": "%8.2f",
            "width": 0.4,
            "position_x": 0.55,
            "position_y": 0.15,
            "title_font_size": 20,
            "label_font_size": 16,
            "n_labels": 0
        },
        annotations=annotations_tensoes
    )


    # Plotar deslocamentos nodais em Z
    plotter = mapdl.result.plot_nodal_displacement(
        position_w_max,
        "UZ",
        lighting=False,
        cpos="iso",
        background="white",
        text_color="black",
        add_text=False,
        show_edges=True,
        edge_color="black",
        cmap="jet",
        # rng=[-2.05, 2.05],
        # rng=[-2.20, 2.05],
        line_width=1.0,
        off_screen=True,
        screenshot=w_dist_img_path,
        show_scalar_bar=True,
        # show_displacement=True,
        show_displacement=False,
        # displacement_factor=20,
        displacement_factor=2,
        scalar_bar_args={
            "title": "Deslocamento em Z (mm)",
            "vertical": False,
            # "fmt": "%8.2f",
            "width": 0.4,
            "position_x": 0.55,
            "position_y": 0.15,
            "title_font_size": 20,
            "label_font_size": 16,
            "n_labels": 0
        },
        annotations=annotations_deslocamentos
    )

In [6]:
analysis_file_name = 'BL1M1P0L0T0_k0_MS25_SP1'
sufixo = 'com_virgula'
analysis_dir = f'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/{analysis_file_name}'
analysis_db_path = f'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/{analysis_file_name}/{analysis_file_name}.db'
flag = 0
annotations_deslocamentos = {
    0.0: "0,00",
    0.70: "0,70",
    1.41: "1,41",
    2.11: "2,11",
    2.82: "2,82"
}
interagir_com_mapdl(mapdl, analysis_dir, analysis_file_name, analysis_db_path, flag, annotations_deslocamentos, sufixo)

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - *** WARNING ***                         CP =       0.062   TIME= 08:21:05
 The working directory specified (D:/constructal_automate_analysis_files/
 ec_2a_2_2_phi_0.3_lancanova_2024/BL1M1P0L0T0_k0_MS25_SP1) is not a      
 directory.  The /CWD command is ignored.


MapdlCommandIgnoredError: *** WARNING ***                         CP =       0.062   TIME= 08:21:05
 The working directory specified (D:/constructal_automate_analysis_files/
 ec_2a_2_2_phi_0.3_lancanova_2024/BL1M1P0L0T0_k0_MS25_SP1) is not a      
 directory.  The /CWD command is ignored.

Ignore these messages by setting 'ignore_errors'=True.


In [80]:
analysis_file_name = 'BL2M1P0.3L4T2_k4.8_MS25_SP68'
sufixo = 'legenda_Nts'
analysis_dir = f'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/{analysis_file_name}'
analysis_db_path = f'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/{analysis_file_name}/{analysis_file_name}.db'
flag = 0
annotations_deslocamentos = {
    -2.05: "-2,05",
    -1.02: "-1,02",
    0.00: "0,00",
    1.02: "1,02",
    2.05: "2,05"
}
interagir_com_mapdl(mapdl, analysis_dir, analysis_file_name, analysis_db_path, flag, annotations_deslocamentos, sufixo)

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Curre

Widget(value='<iframe src="http://localhost:59788/index.html?ui=P_0x223b74ebe90_77&reconnect=auto" class="pyvi…

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = 

Widget(value='<iframe src="http://localhost:59788/index.html?ui=P_0x223b74ee120_78&reconnect=auto" class="pyvi…

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = 

Widget(value='<iframe src="http://localhost:59788/index.html?ui=P_0x223b74ecb60_79&reconnect=auto" class="pyvi…

In [82]:
analysis_file_name = 'BL2M1P0.3L4T2_k4.8_MS25_SP68'
sufixo = 'legenda_Nls'
analysis_dir = f'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/{analysis_file_name}'
analysis_db_path = f'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/{analysis_file_name}/{analysis_file_name}.db'
flag = 0
annotations_deslocamentos = {
    -2.20: "-2,20",
    -1.14: "-1,14",
    -0.08: "-0,08",
    0.99: "0,99",
    2.05: "2,05"
}
interagir_com_mapdl(mapdl, analysis_dir, analysis_file_name, analysis_db_path, flag, annotations_deslocamentos, sufixo)

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Curre

Widget(value='<iframe src="http://localhost:59788/index.html?ui=P_0x223b74daab0_80&reconnect=auto" class="pyvi…

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = 

Widget(value='<iframe src="http://localhost:59788/index.html?ui=P_0x223b74edf70_81&reconnect=auto" class="pyvi…

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k4.8_MS25_SP68
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = 

Widget(value='<iframe src="http://localhost:59788/index.html?ui=P_0x223b74f3320_82&reconnect=auto" class="pyvi…

In [86]:
analysis_file_name = 'BL2M1P0L0T0_k0_MS25_SP1'
sufixo = 'legenda_virgula'
analysis_dir = f'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/{analysis_file_name}'
analysis_db_path = f'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/{analysis_file_name}/{analysis_file_name}.db'
flag = 1
annotations_deslocamentos = {
    0.00: "0,00",
    9.51: "9,51",
    19.03: "19,03",
    28.54: "28,54",
    38.05: "38,05"
}
interagir_com_mapdl(mapdl, analysis_dir, analysis_file_name, analysis_db_path, flag, annotations_deslocamentos, sufixo)

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0L0T0_k0_MS25_SP1
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0L0T0_k0_MS25_SP1
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0L0T0_k0_MS25_SP1
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0L0T0_k0_MS25_SP1
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0L0T0_k0_MS25_SP1
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory swit

Widget(value='<iframe src="http://localhost:59788/index.html?ui=P_0x22408787020_86&reconnect=auto" class="pyvi…

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0L0T0_k0_MS25_SP1
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0L0T0_k0_MS25_SP1
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0L0T0_k0_MS25_SP1
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0L0T0_k0_MS25_SP1
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0L0T0_k0_MS25_SP1
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0L0T0_k0_MS25_SP1
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0L0T0_k0_MS25_SP1
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0L0T0_k0_MS25_SP1
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl

Widget(value='<iframe src="http://localhost:59788/index.html?ui=P_0x22408787f20_87&reconnect=auto" class="pyvi…

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0L0T0_k0_MS25_SP1
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0L0T0_k0_MS25_SP1
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0L0T0_k0_MS25_SP1
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0L0T0_k0_MS25_SP1
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0L0T0_k0_MS25_SP1
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0L0T0_k0_MS25_SP1
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0L0T0_k0_MS25_SP1
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0L0T0_k0_MS25_SP1
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl

Widget(value='<iframe src="http://localhost:59788/index.html?ui=P_0x224087873e0_88&reconnect=auto" class="pyvi…

In [9]:
analysis_file_name = 'BL2M1P0.3L2T2_k56.2_MS25_SP2'
sufixo = 'legenda_virgula'
analysis_dir = f'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/{analysis_file_name}'
analysis_db_path = f'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/{analysis_file_name}/{analysis_file_name}.db'
flag = 1
annotations_deslocamentos = {
    -0.156: "-0,16",
    -0.12: "-0,12",
    -0.07: "-0,07",
    -0.03: "-0,03",
    0.01: "0,01",
}
interagir_com_mapdl(mapdl, analysis_dir, analysis_file_name, analysis_db_path, flag, annotations_deslocamentos, sufixo)

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L2T2_k56.2_MS25_SP2
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - CURRENT JOBNAME REDEFINED AS BL2M1P0.3L2T2_k56.2_MS25_SP2
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - RESUME ANSYS DATA FROM FILE NAME=BL2M1P0.3L2T2_k56.2_MS25_SP2.db                                                                                                                                                                                                                                     

 *** MAPDL GLOBAL STATUS *** 

 TITLE = BL2M1P0.3L2T2_k56.2_MS25_SP2                                          
 ANALYSIS TYPE =  STATIC (STEADY-STATE)
        28 KEYPOINTS DEFINED     MAX KEYPOINT NUMBER =             28
        48 LINES DEFINED         MAX LINE NUMBER =                 60
        21 AREAS DEFINED         MAX AREA NUMBER =                 

Widget(value='<iframe src="http://localhost:50124/index.html?ui=P_0x1e556ea1a60_6&reconnect=auto" class="pyvis…

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L2T2_k56.2_MS25_SP2
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_2_2_phi_0.3_lancanova_2024\BL2M1P0.3L2T2_k56.2_MS25_SP2


Widget(value='<iframe src="http://localhost:50124/index.html?ui=P_0x1e566b8d220_7&reconnect=auto" class="pyvis…

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L2T2_k56.2_MS25_SP2
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_2_2_phi_0.3_lancanova_2024\BL2M1P0.3L2T2_k56.2_MS25_SP2


Widget(value='<iframe src="http://localhost:50124/index.html?ui=P_0x1e572d9e120_8&reconnect=auto" class="pyvis…

In [14]:
analysis_file_name = 'BL2M1P0.3L5T5_k0.375_MS25_SP129'
sufixo = 'legenda_virgula'
analysis_dir = f'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/{analysis_file_name}'
analysis_db_path = f'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/{analysis_file_name}/{analysis_file_name}.db'
flag = 1
annotations_deslocamentos = {
    0.00: "0,00",
    10.24: "10,24",
    20.47: "20,47",
    30.71: "30,71",
    40.94: "40,94",
}
interagir_com_mapdl(mapdl, analysis_dir, analysis_file_name, analysis_db_path, flag, annotations_deslocamentos, sufixo)

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - CURRENT JOBNAME REDEFINED AS BL2M1P0.3L5T5_k0.375_MS25_SP129
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - RESUME ANSYS DATA FROM FILE NAME=BL2M1P0.3L5T5_k0.375_MS25_SP129.db                                                                                                                                                                                                                                  

 *** MAPDL GLOBAL STATUS *** 

 TITLE = BL2M1P0.3L5T5_k0.375_MS25_SP129                                       
 ANALYSIS TYPE =  STATIC (STEADY-STATE)
        94 KEYPOINTS DEFINED     MAX KEYPOINT NUMBER =             94
       189 LINES DEFINED         MAX LINE NUMBER =                213
        96 AREAS DEFINED         MAX AREA NUMBER =                107
         1 ELEM TYPES DEFINED    MAX ELEM TYPE NUMBER =             1
      4158 ELEMENTS DEFINED      MAX ELEMENT NUMBER =            4158
     12697 NODES DEFINED         MAX NODE N

Widget(value='<iframe src="http://localhost:50124/index.html?ui=P_0x1e50ca49a90_15&reconnect=auto" class="pyvi…

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L5T5_k0.375_MS25_SP129
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_2_2_phi_0.3_lancanova_2024\BL2M1P0.3L5T5_k0.375_MS25_SP129


Widget(value='<iframe src="http://localhost:50124/index.html?ui=P_0x1e50ca4b320_16&reconnect=auto" class="pyvi…

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L5T5_k0.375_MS25_SP129
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_2_2_phi_0.3_lancanova_2024\BL2M1P0.3L5T5_k0.375_MS25_SP129


Widget(value='<iframe src="http://localhost:50124/index.html?ui=P_0x1e509638710_17&reconnect=auto" class="pyvi…

In [15]:
analysis_file_name = 'BL2M1P0.3L3T2_k5.466_MS25_SP36'
sufixo = 'legenda_virgula'
analysis_dir = f'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/{analysis_file_name}'
analysis_db_path = f'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/{analysis_file_name}/{analysis_file_name}.db'
flag = 1
annotations_deslocamentos = {
    -1.21: "-1,21",
    -0.54: "-0,54",
    0.14: "0,14",
    0.81: "0,81",
    1.48: "1,48",
}
interagir_com_mapdl(mapdl, analysis_dir, analysis_file_name, analysis_db_path, flag, annotations_deslocamentos, sufixo)

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L5T5_k0.375_MS25_SP129
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - CURRENT JOBNAME REDEFINED AS BL2M1P0.3L3T2_k5.466_MS25_SP36
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - RESUME ANSYS DATA FROM FILE NAME=BL2M1P0.3L3T2_k5.466_MS25_SP36.db                                                                                                                                                                                                                                   

 *** MAPDL GLOBAL STATUS *** 

 TITLE = BL2M1P0.3L3T2_k5.466_MS25_SP36                                        
 ANALYSIS TYPE =  STATIC (STEADY-STATE)
        36 KEYPOINTS DEFINED     MAX KEYPOINT NUMBER =             36
        64 LINES DEFINED         MAX LINE NUMBER =                 78
        29 AREAS DEFINED         MAX AREA NUMBER =            

Widget(value='<iframe src="http://localhost:50124/index.html?ui=P_0x1e509536db0_18&reconnect=auto" class="pyvi…

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L3T2_k5.466_MS25_SP36
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_2_2_phi_0.3_lancanova_2024\BL2M1P0.3L3T2_k5.466_MS25_SP36


Widget(value='<iframe src="http://localhost:50124/index.html?ui=P_0x1e50c29d160_19&reconnect=auto" class="pyvi…

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L3T2_k5.466_MS25_SP36
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_2_2_phi_0.3_lancanova_2024\BL2M1P0.3L3T2_k5.466_MS25_SP36


Widget(value='<iframe src="http://localhost:50124/index.html?ui=P_0x1e50c227fb0_20&reconnect=auto" class="pyvi…

In [18]:
mapdl.exit()

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_1a_phi_0.3_lancanova_2024\BL1M1P0L0T0_k0_MS25_SP1
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_1a_phi_0.3_lancanova_2024\BL1M1P0L0T0_k0_MS25_SP1
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_1a_phi_0.3_lancanova_2024\BL1M1P0L0T0_k0_MS25_SP1
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_1a_phi_0.3_lancanova_2024\BL1M1P0L0T0_k0_MS25_SP1


In [59]:
# analysis_file_name = 'BL2M1P0.3L2T2_k56.2_MS25_SP2'
# analysis_file_name = 'BL2M1P0.3L4T2_k4.8_MS25_SP68'
# analysis_file_name = 'BL2M1P0.3L3T2_k5.466_MS25_SP36'
# analysis_file_name = 'BL2M1P0.3L5T5_k0.375_MS25_SP129'
analysis_file_name = 'BL2M1P0.3L3T5_k0.425_MS25_SP65'
analysis_dir = f'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/{analysis_file_name}'
analysis_db_path = f'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/{analysis_file_name}/{analysis_file_name}.db'
flag = 0
interagir_com_mapdl_deformado(mapdl, analysis_dir, analysis_file_name, analysis_db_path, flag)

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L5T5_k0.375_MS25_SP129
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - CURRENT JOBNAME REDEFINED AS BL2M1P0.3L5T5_k0.375_MS25_SP129
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - RESUME ANSYS DATA FROM FILE NAME=BL2M1P0.3L5T5_k0.375_MS25_SP129.db                                                                                                                                                                                                                                  

 *** MAPDL GLOBAL STATUS *** 

 TITLE = BL2M1P0.3L5T5_k0.375_MS25_SP129                                       
 ANALYSIS TYPE =  STATIC (STEADY-STATE)
        94 KEYPOINTS DEFINED     MAX KEYPOINT NUMBER =             94
       189 LINES DEFINED         MAX LINE NUMBER =                213
        96 AREAS DEFINED         MAX AREA NUMBER =           

Widget(value='<iframe src="http://localhost:53039/index.html?ui=P_0x2be938875f0_90&reconnect=auto" class="pyvi…

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L5T5_k0.375_MS25_SP129
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_2_2_phi_0.3_lancanova_2024\BL2M1P0.3L5T5_k0.375_MS25_SP129


Widget(value='<iframe src="http://localhost:53039/index.html?ui=P_0x2be93887f80_91&reconnect=auto" class="pyvi…

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L5T5_k0.375_MS25_SP129
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_2_2_phi_0.3_lancanova_2024\BL2M1P0.3L5T5_k0.375_MS25_SP129


Widget(value='<iframe src="http://localhost:53039/index.html?ui=P_0x2bea6c76780_92&reconnect=auto" class="pyvi…

In [17]:
analysis_file_name = 'BL1M1P0L0T0_k0_MS25_SP1'
analysis_base_dir = 'ec_1a_phi_0.3_lancanova_2024'
sufixo = 'com_virgula'
analysis_dir = f'D:/constructal_automate_analysis_files/{analysis_base_dir}/{analysis_file_name}'
analysis_db_path = f'D:/constructal_automate_analysis_files/{analysis_base_dir}/{analysis_file_name}/{analysis_file_name}.db'
flag = 1
annotations_deslocamentos = {
    -17.73: "-17,74",
    -8.77: "-8,77",
    0.19: "0,19",
    9.15: "9,15",
    18.11: "18,11"
}
interagir_com_mapdl(mapdl, analysis_dir, analysis_file_name, analysis_db_path, flag, annotations_deslocamentos, sufixo)

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_1a_phi_0.3_lancanova_2024/BL1M1P0L0T0_k0_MS25_SP1
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_1a_phi_0.3_lancanova_2024/BL1M1P0L0T0_k0_MS25_SP1
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - CURRENT JOBNAME REDEFINED AS BL1M1P0L0T0_k0_MS25_SP1
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - CURRENT JOBNAME REDEFINED AS BL1M1P0L0T0_k0_MS25_SP1
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - RESUME ANSYS DATA FROM FILE NAME=BL1M1P0L0T0_k0_MS25_SP1.db                                                                                                                                                                                                                                          

 *** MAPDL GLOBAL STATUS *** 

 TITLE = BL1M1P0L0T0_k0_MS25_SP1                          

Widget(value='<iframe src="http://localhost:53501/index.html?ui=P_0x1d4838e1490_15&reconnect=auto" class="pyvi…

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL1M1P0L0T0_k0_MS25_SP1
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL1M1P0L0T0_k0_MS25_SP1
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_1a_phi_0.3_lancanova_2024\BL1M1P0L0T0_k0_MS25_SP1
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_1a_phi_0.3_lancanova_2024\BL1M1P0L0T0_k0_MS25_SP1


Widget(value='<iframe src="http://localhost:53501/index.html?ui=P_0x1d4891754f0_16&reconnect=auto" class="pyvi…

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL1M1P0L0T0_k0_MS25_SP1
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL1M1P0L0T0_k0_MS25_SP1
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_1a_phi_0.3_lancanova_2024\BL1M1P0L0T0_k0_MS25_SP1
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_1a_phi_0.3_lancanova_2024\BL1M1P0L0T0_k0_MS25_SP1


Widget(value='<iframe src="http://localhost:53501/index.html?ui=P_0x1d4891767e0_17&reconnect=auto" class="pyvi…